## AI 全栈工程师课程 Fine-tuning 实验指导


## 搭建环境

我们选择 AutoDL 的 4090 云 GPU 做为标准实验环境。因为独立部署自己训练的大模型， 4090 是目前性价比最高的方案。

如果要大规模部署，那还是推荐阿里云等大型云服务商的 V100 等 GPU 云主机。阿里云提供免费试用，可以到[这里申请](https://free.aliyun.com/?product=9602825&spm=5176.28055625.J_5831864660.9.e939154aYoM8ST&scm=20140722.M_9553144.P_154.MO_1802-ID_9553144-MID_9553144-CID_20080-ST_7663-V_1)。

### 第一步：注册 AutoDL 官网并实名认证

官网地址：https://www.autodl.com/

按常规方法注册、登录后，还要做[个人实名认证](https://www.autodl.com/console/center/account/safe)。

如果不实名认证，无法从浏览器访问你训练的大模型，看不到训练效果。


### 第二步：兑换余额

找班班获取兑换码，然后进入[控制台](https://www.autodl.com/console/homepage/personal)，兑换代金券，获取我们赠送的额度

![兑换卷](./coupon_input.png)

兑换成功会显示如下界面

![兑换成功](./coupon_success.png)

如果获得的额度没能完成实验，可以联系班班获取更多的额度。


### 第三步：挑选 GPU

点击[算力市场](https://www.autodl.com/market/list)，挑选 GPU。

RTX 4090，显存大于 20GB 的均可。选中后，点击「n 卡可租」。

![创建容器](./choose_gpu.png)


### 第四步：创建容器

只需选择一个 GPU，然后点击「社区镜像」，搜索 `agiclass/fine-tuning-lab/finetune-lab-v8` ，选择最新版，然后点击「立即创建」

![创建容器](./create_docker.png)

![选择镜像](./image_select.png)

在[容器实例](https://www.autodl.com/console/instance/list)中可以看到刚创建的容器。当「状态」为「运行中」时，表示创建完成，并且开始计费。

如果想暂停计费，请点击「关机」。下次需要使用时，再点击「开机」。

`agiclass/fine-tuning-lab/finetune-lab-v8` 是我们制作的容器镜像，其中配置好了实验所需依赖包环境，提供了训练好了的checkpoints。

## 准备实验环境

通过这个过程，检验环境是否搭建正确。

### 第五步：ssh 登录容器

在自己电脑的命令行工具内，通过 ssh 登录容器。可以从[容器实例](https://www.autodl.com/console/instance/list)页面直接复制 ssh 命令和密码。

![控制台](ssh_copy.png)

第一次执行命令，会询问是否信任主机，输入 yes 即可。

![ssh](ssh_login.png)

出现下面界面，说明登录成功。

![ssh](./login_success.png)


### 第六步：实验环境准备

实验环境所需的python依赖包已安装好，后续需要将实验所需的预训练模型权重从huggingface或modelscope下载到本地

拉取qwen2-7b的权重

```bash
cd /root/autodl-tmp/
git clone https://www.modelscope.cn/qwen/Qwen2-7B-Instruct.git
cd Qwen2-7B-Instruct
# 清除掉.git目录，避免数据盘空间不足
rm -rf .git 
```

glm4-9b在modelscope中有托管(要使用特定版本，最新的checkpoint有问题，会导致显存溢出)，可以使用git-lfs来同步到本地数据盘（耐心等待十几分钟）

```bash
cd /root/autodl-tmp
git lfs install
GIT_LFS_SKIP_SMUDGE=1 git clone https://www.modelscope.cn/ZhipuAI/glm-4-9b-chat.git
cd glm-4-9b-chat
git checkout eeb4a6d04dd8aa25a6c46edd0837792f8bc64e3a
git lfs fetch --include="tokenizer.model"
git lfs checkout tokenizer.model
git lfs fetch --include="*.safetensors"
git lfs checkout *.safetensors
# 清理掉.git目录减少数据盘空间占用
rm -rf .git
```

<div class="alert alert-warning">
<b>注意：</b> 要给glm4-9b的modeling_chatglm.py文件打个补丁修改一下才能在较新版本的transformers库中使用
</div>

```bash
sed -i '815s/.*/        cache_name, cache = self._extract_past_from_model_output(outputs)/; 816s/.*/        model_kwargs[cache_name] = cache/; 817d' /root/autodl-tmp/glm-4-9b-chat/modeling_chatglm.py
```

同理可拉取llama3.1-8b的权重

```bash
cd /root/autodl-tmp/
git clone https://www.modelscope.cn/LLM-Research/meta-llama-3.1-8b-instruct.git
cd meta-llama-3.1-8b-instruct
# 清除掉.git目录，避免数据盘空间不足
rm -rf .git 
```

<div class="alert alert-warning">
<b>注意：</b>如果出现数据盘容量不足的问题时，可以删除预训练模型目录中的.git目录，当然最好是付费扩容数据盘，价格也很便宜的
</div>

finetune的代码托管在github上，大家可以去同步拉取(也欢迎大家来star噢)，地址是 https://github.com/agiclass/fine-tuning-lab 代码在 v8 分支上。从github拉取finetune代码，如网络不稳定导致拉取失败，可以手动重复尝试

```bash
cd /root/autodl-tmp/
# 如果网络不畅通可以使用ghproxy来代理从github同步代码
# 或从实验室中拿到的已打包好的代码包然后手动传到autodl服务器上
git clone https://mirror.ghproxy.com/https://github.com/agiclass/fine-tuning-lab -b v8
```

然后到web demo的目录加载并运行模型

```bash
cd /root/autodl-tmp/fine-tuning-lab/web_demo/
bash qwen2_lora.sh
## 其他的也是同理
# bash glm4_qlora.sh
# bash llama3_qlora.sh
```

出现下面截图表示web服务启动成功

![启动webui](./webui-start.png)

### 第七步：从浏览器访问

通过autodl平台提供的“自定义服务”功能访问web demo网页，参考页面上提示的使用方法

![custom-http](./custom-http-start.png)

然后就可以在这个“访问”按钮跳转的链接中访问到web网页了，地址类似这样：`https://u202774-8229-2a7abb05.westb.seetacloud.com:8443/`

<span style="background-color: LightGray; color: black;">
由于安全限制，autodl平台有时会关闭http服务访问，这种情况下需要使用ssh通道转发回本地访问，
具体使用方法参考“自定义服务”弹窗页面中的提示，根据自己电脑操作系统来选相应的命令
方式类似于上一步登陆时的ssh命令，添加 `-L 6006:localhost:6006`：
</span>

```bash
# 比如上一步从autodl控制台上复制下来的ssh命令是：
# ssh -p 17844 root@connect.westb.seetacloud.com
# 则这一步在前面加上 -L 6006:localhost:6006
# ！！！ 注意把这里的地址和端口换成你自己的 ！！！
ssh -L 6006:localhost:6006 -p 17844 root@connect.westb.seetacloud.com
```

<span style="background-color: LightGray; color: black;">
然后就可以在**自己的浏览器**里面访问 http://localhost:6006 的地址访问webui demo了
</span>

等待页面加载完成，就可以和自己运行的大模型对话了。

---

可以尝试输入以下问询的输入如

“订一个七八百元的酒店”

![webui](./webui-gui-1.png)

## 开始训练

基于我们在 `/root/autodl-tmp/fine-tuning-lab/data` 目录下提供的数据，用 lora 方法进行微调训练。

执行下面的命令，开始训练，耗时需要几个小时。这期间要保持网络连接，否则可能会导致前台训练任务进程被kill从而训练失败，建议使用tmux/screen等终端复用工具开启后台训练

```bash
# 开启tmux，退出tmux时可以直接关掉终端窗口，或者按快捷键执行detach
# detach快捷键为：ctrl-b d (注意这是先一起按ctrl键和b键，然后再按一下d键)
tmux
cd qwen2
bash train.sh

# detach后或关掉终端再次登录时，可以执行tmux ls命令查看后台的终端
tmux ls
# 选中其中的session编号如0号并attach进去继续使用
tmux a -t 0
# 关于tmux的更多使用方法建议搜一些教程了解
```

只要没有出错信息，没有回到命令输入状态，有滚动输出的内容，有数字、进度条在变化，就说明训练在正常进行。请耐心等待。

训练过程中会有运行的进度条

![训练中](./training-progress.png)

可以使用tensorboard工具在`/root/autodl-tmp/fine-tuning-lab/qwen2/output/`目录将记录的loss可视化出来，运行下面命令

```bash
tensorboard --logdir=<your dir> --bind_all
```

![tensorboard](./tensorboard-start.png)

即可通过上一步同样的自定义服务的那个web页面来查看了，效果如下

![tensorboard](./tensorboard-gui.png)

当前面训练界面的进度条走完，说明训练完成了！

训练生成的文件在 `/root/autodl-tmp/fine-tuning-lab/qwen2/output` 目录下。`checkpoint-nnn` 是训练过程中的中间结果，`nnn` 数字最大的那个是最终结果。


## 测试训练后的模型

执行下面命令，默认加载的是课程提供的已训好的模型checkpoint，可以修改其中的路径换成上一步自己训的

![训练完成](./evaluate-checkpoint.png)

```
cd /root/autodl-tmp/fine-tuning-lab/qwen2
bash eval.sh
```

会在测试集上运行完推理，并计算出SLOT和BLEU的指标

然后重复前面**第七步**，也记得将其中checkpoint路径换成自己刚刚训练好的，从浏览器访问你亲手训练的大模型，与之对话，测试各种问题的效果。

![训练完成](./webui-checkpoint.png)

<div class="alert alert-warning">
<b>注意：</b>完成实验后不要忘记<a href="https://www.autodl.com/console/instance/list" target="_blank">关机</a>，以免继续产生费用。
</div>


## [Optional] 基于llama3.1-8b或glm4-9b的训练

训练过程跟前文所讲训练 qwen2 基本一样，但使用了 qlora 训练方法，执行下面的命令，开始训练。

```bash
cd llama3
bash train.sh
```

## FAQ

### 如果需要访问科学的彼岸

AutoDL 的服务器是在国内，如果需要从它访问国外的网站，可以执行下面的命令：

```bash
source /etc/network_turbo
```